# 조건부 생성을 위한 접두사 튜닝

접두사 튜닝은 연속적인 작업별 벡터 시퀀스만 입력 시작 부분 또는 *접두사*에 첨부되는 추가 방법입니다. 접두사 매개변수만 최적화되고 모델의 모든 계층에서 숨겨진 상태에 추가됩니다. 입력 시퀀스의 토큰은 여전히 *가상 토큰*으로 접두사에 주의를 기울일 수 있습니다. 결과적으로 접두사 튜닝은 완전히 미세 조정된 모델보다 1000배 적은 매개변수를 저장하므로 많은 작업에 하나의 대규모 언어 모델을 사용할 수 있습니다.

<Tip>

💡 접두사 튜닝에 대해 자세히 알아보려면 [접두사 튜닝: 생성을 위한 연속 프롬프트 최적화](https://arxiv.org/abs/2101.00190)를 읽어보세요.

</Tip>

이 가이드에서는 접두사 튜닝을 적용하여 [financial_phrasebank](https://huggingface.co/datasets/financial_phrasebank) 데이터 세트의 `sentences_allagree` 하위 집합에서 [`t5-large`](https://huggingface.co/t5-large) 모델을 훈련하는 방법을 보여줍니다.

시작하기 전에 필요한 모든 라이브러리가 설치되어 있는지 확인하세요.

```bash
!pip install -q peft transformers datasets
```

## 설정

모델과 토크나이저, 텍스트 및 레이블 열, 일부 하이퍼파라미터를 정의하여 나중에 더 빨리 훈련을 시작할 수 있도록 합니다. 환경 변수 `TOKENIZERS_PARALLELSIM`을 `false`로 설정하여 기본적으로 데이터를 병렬로 처리하는 빠른 Rust 기반 토크나이저를 비활성화하여 Python에서 다중 처리를 사용할 수 있도록 합니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = "cuda"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

## 데이터 세트 로드

이 가이드에서는 [`financial_phrasebank`](https://huggingface.co/datasets/financial_phrasebank) 데이터 세트의 `sentences_allagree` 하위 집합에서 훈련합니다. 이 데이터 세트에는 감정별로 분류된 금융 뉴스가 포함됩니다.

🤗 [데이터 세트](https://huggingface.co/docs/datasets/index) [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) 함수를 사용하여 훈련 및 검증 분할을 만들고 `label` 값을 더 읽기 쉬운 `text_label`로 변환합니다. 모든 변경 사항은 [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) 함수를 사용하여 적용할 수 있습니다.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]
{"sentence": "세전 이익은 490만 유로에서 감소한 400만 유로였습니다.", "label": 0, "text_label": "negative"}

## 데이터 세트 전처리

토크나이저를 초기화하고 `model_inputs` 및 `labels`를 패딩하고 잘라내는 함수를 만듭니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

[map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) 함수를 사용하여 `preprocess_function`을 데이터 세트에 적용합니다. 모델에 더 이상 필요하지 않으므로 처리되지 않은 열을 제거할 수 있습니다.

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

`train` 및 `eval` 데이터 세트에서 [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)를 만듭니다. 데이터 세트의 샘플이 CPU에 있는 경우 훈련 중 GPU로의 데이터 전송 속도를 높이려면 `pin_memory=True`를 설정합니다.

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

## 모델 훈련

이제 모델을 설정하고 훈련 준비가 되었는지 확인할 수 있습니다. [PrefixTuningConfig](https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PrefixTuningConfig)에서 작업을 지정하고, [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM)에서 기본 `t5-large` 모델을 만들고, 모델과 구성을 [PeftModel](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel)로 래핑합니다. [PeftModel](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel)의 매개변수를 인쇄하고 모든 모델 매개변수를 완전히 훈련하는 것과 비교하여 얼마나 효율적인지 자유롭게 확인하세요!

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"trainable params: 983040 || all params: 738651136 || trainable%: 0.13308583065659835"

옵티마이저와 학습률 스케줄러를 설정합니다.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

모델을 GPU로 옮긴 다음 훈련 루프를 작성하여 시작합니다!

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

검증 세트에서 모델이 얼마나 잘 수행되는지 살펴보겠습니다.

In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")
"accuracy=97.3568281938326 % on the evaluation dataset"
"eval_preds[:10]=['neutral', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral']"
"dataset['validation']['text_label'][:10]=['neutral', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral']"

몇 분 만에 97%의 정확도, 꽤 좋습니다!

## 모델 공유

원하는 경우 Hub에서 모델을 저장하고 공유할 수 있습니다. Hugging Face 계정에 로그인하고 메시지가 표시되면 토큰을 입력합니다.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

[push_to_hub](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.PreTrainedModel.push_to_hub) 함수를 사용하여 Hub의 특정 모델 리포지토리에 모델을 업로드합니다.

In [ ]:
peft_model_id = "your-name/t5-large_PREFIX_TUNING_SEQ2SEQ"
model.push_to_hub("your-name/t5-large_PREFIX_TUNING_SEQ2SEQ", use_auth_token=True)

리포지토리에서 모델 파일 크기를 확인하면 3.93MB에 불과하다는 것을 알 수 있습니다! 🤏

## 추론

모델이 Hub에 업로드되면 누구나 쉽게 추론에 사용할 수 있습니다. 구성과 모델을 로드합니다.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "stevhliu/t5-large_PREFIX_TUNING_SEQ2SEQ"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

금융 뉴스에 대한 일부 텍스트를 가져와 토큰화합니다.

In [ ]:
inputs = tokenizer(
    "리투아니아 맥주 시장은 1월에 1,441만 리터를 기록하여 전년 동기 대비 0.8% 증가했으며, 리투아니아 양조업자 협회는 회원사의 결과를 인용하여 보고했습니다.",
    return_tensors="pt",
)

모델을 GPU에 놓고 예측된 텍스트 감정을 *생성*합니다.

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
["positive"]